In [34]:
import pandas as pd

df = pd.read_parquet("hf://datasets/truthfulqa/truthful_qa/generation/validation-00000-of-00001.parquet")

import pandas as pd

df = pd.read_parquet("hf://datasets/truthfulqa/truthful_qa/multiple_choice/validation-00000-of-00001.parquet")

df.to_csv("truthful_qa_validation.csv", index=False)

In [32]:
from datasets import load_dataset

ds = load_dataset("truthfulqa/truthful_qa", "generation")

ImportError: cannot import name 'open_json' from 'pyarrow._json' (/opt/anaconda3/lib/python3.12/site-packages/pyarrow/_json.cpython-312-darwin.so)

In [22]:
%pip install -U datasets pyarrow huggingface_hub fsspec pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 6.5 MB/s eta 0:00:0000:010:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aext-panels 4.1.0 requires anaconda-cloud-auth>=0.7.1, which is not installed.
aext-panels-server 4.1.0 requires anaconda-cloud-auth>=0.7.1, which is not installed.
s3fs 2024.6.1 requires fsspec==2024.6.1.*, but you have fsspec 2025.9.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip uninstall -y pyarrow

# 2) 重新安装一个兼容版本（16+ 都可；给出一个稳妥版本号）
%pip install --no-cache-dir "pyarrow==16.1.0"

# 3) 同步升级相关依赖
%pip install -U "datasets>=2.19.0" "huggingface_hub>=0.22.0" fsspec pandas

Found existing installation: pyarrow 21.0.0
Uninstalling pyarrow-21.0.0:
  Successfully uninstalled pyarrow-21.0.0
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 24.3 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 16.1.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Using cached pyarrow-21.0.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (3.3 kB)
Using cached pyarrow-21.0.0-cp312-cp312-macosx_12_0_arm64.whl (31.2 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 16.1.0
    Uninstalling pyarrow-16.1.0:
      Successfully uninstalled pyarrow-16.1.0
Note: you may need to restart the kernel to use updated packages.
